In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import string

### For some reason, there are broken table tags in a lot of the tables below when they're rendered on Github. They don't actually look like that

In [2]:
# df = pd.read_csv('data/JEOPARDY_CSV.csv', encoding='utf-8')
full_df = pd.read_csv('../data/JEOPARDY_CSV.csv', encoding='utf-8')
print full_df.shape
full_df.head()

(216930, 7)


,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [3]:
df = full_df.sample(frac=0.01)
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4666,2004-12-13,Jeopardy!,DON WE NOW OUR GUY APPAREL,$200,"The 2 numbers on a man's dress shirt, 16/34 fo...",neck size and sleeve length
1,5549,2008-10-23,Jeopardy!,VERBS,$1000,This verb means to use another's work without ...,to pirate
2,6238,2011-11-02,Jeopardy!,PAY BALL!,$600,"This player nicknamed ""Big Papi"" pulls down $1...",David Ortiz
3,4657,2004-11-30,Double Jeopardy!,FUNNY HATS,$2000,"In Wagner's operas, this eldest Valkyrie is st...",Brunhilde
4,2873,1997-02-12,Jeopardy!,WOMEN AUTHORS,$200,Kim Wozencraft & Dorothy Uhnak used their expe...,Policewomen


In [5]:
# Remove the dumb spaces
df.columns = ['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer']

# Convert to Datetime
df['Air Date'] = pd.to_datetime(df['Air Date'])

# Clean out Value column
df['Value'] = df['Value'].str.replace('$','')
df['Value'] = df['Value'].str.replace(',','')
df['Value'] = df['Value'].apply(lambda x: None if x == 'None' else int(x))
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4666,2004-12-13,Jeopardy!,DON WE NOW OUR GUY APPAREL,200.0,"The 2 numbers on a man's dress shirt, 16/34 fo...",neck size and sleeve length
1,5549,2008-10-23,Jeopardy!,VERBS,1000.0,This verb means to use another's work without ...,to pirate
2,6238,2011-11-02,Jeopardy!,PAY BALL!,600.0,"This player nicknamed ""Big Papi"" pulls down $1...",David Ortiz
3,4657,2004-11-30,Double Jeopardy!,FUNNY HATS,2000.0,"In Wagner's operas, this eldest Valkyrie is st...",Brunhilde
4,2873,1997-02-12,Jeopardy!,WOMEN AUTHORS,200.0,Kim Wozencraft & Dorothy Uhnak used their expe...,Policewomen


In [6]:
df['Question'].value_counts()[0:5]

He explained his "Decision Points" in the 2010 book of that name                                                 1
As a boy in Kansas, he said his dream was to be a Major League Baseball player; he went into the Army instead    1
Remixes from her "Post" album are featured on this Icelandic pop star's 1996 album "Telegram"                    1
The name of this third male human to be mentioned in Genesis means "breath" in Hebrew                            1
Gentile da Fabriano used the international Gothic style for his painting "The Adoration Of" this trio            1
Name: Question, dtype: int64

In [7]:
df.shape

(2169, 7)

In [8]:
# Drop some useless questions
df = df[df['Question'] != '[audio clue]']
df = df[df['Question'] != '[video clue]']
df = df[df['Question'] != '[filler]']
df = df[df['Question'] != '(audio clue)']

In [9]:
df.shape

(2168, 7)

In [10]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
print len(stopwords)
print stopwords[:10]

153
[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your']


In [11]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [12]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [13]:
def create_clean_columns(df):    
    df['clean_question'] = df['Question'].apply(cleanhtml)
    df['clean_answer'] = df['Answer'].apply(cleanhtml)
    df['clean_category'] = df['Category'].apply(cleanhtml)
    df['everything'] = df['clean_question']+' '+df['clean_answer']+' '+df['clean_category']
    return df

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', '')
#     cleantext = cleantext.translate(None, string.punctuation)
#     cleantext = cleantext.replace('\'', '')
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    cleantext = regex.sub('', cleantext)
    cleantext = cleantext.lower()
    return cleantext

In [14]:
create_clean_columns(df)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_category,everything
0,4666,2004-12-13,Jeopardy!,DON WE NOW OUR GUY APPAREL,200.0,"The 2 numbers on a man's dress shirt, 16/34 fo...",neck size and sleeve length,the 2 numbers on a mans dress shirt 1634 for e...,neck size and sleeve length,don we now our guy apparel,the 2 numbers on a mans dress shirt 1634 for e...
1,5549,2008-10-23,Jeopardy!,VERBS,1000.0,This verb means to use another's work without ...,to pirate,this verb means to use anothers work without p...,to pirate,verbs,this verb means to use anothers work without p...
2,6238,2011-11-02,Jeopardy!,PAY BALL!,600.0,"This player nicknamed ""Big Papi"" pulls down $1...",David Ortiz,this player nicknamed big papi pulls down 125 ...,david ortiz,pay ball,this player nicknamed big papi pulls down 125 ...
3,4657,2004-11-30,Double Jeopardy!,FUNNY HATS,2000.0,"In Wagner's operas, this eldest Valkyrie is st...",Brunhilde,in wagners operas this eldest valkyrie is ster...,brunhilde,funny hats,in wagners operas this eldest valkyrie is ster...
4,2873,1997-02-12,Jeopardy!,WOMEN AUTHORS,200.0,Kim Wozencraft & Dorothy Uhnak used their expe...,Policewomen,kim wozencraft dorothy uhnak used their exper...,policewomen,women authors,kim wozencraft dorothy uhnak used their exper...
5,968,1988-11-16,Double Jeopardy!,FAMOUS WOMEN,400.0,In 1955 this miniskirt innovator opened her fi...,Mary Quant,in 1955 this miniskirt innovator opened her fi...,mary quant,famous women,in 1955 this miniskirt innovator opened her fi...
6,4646,2004-11-15,Jeopardy!,HORNED ANIMALS,800.0,This North American animal's population had dr...,the buffalo (or the bison),this north american animals population had dro...,the buffalo or the bison,horned animals,this north american animals population had dro...
7,4031,2002-02-25,Double Jeopardy!,BEHIND THE MUSIC,800.0,"<a href=""http://www.j-archive.com/media/2002-0...",Grease,this 50s high school love story thats thrilled...,grease,behind the music,this 50s high school love story thats thrilled...
8,4455,2004-01-09,Jeopardy!,RITE ON!,400.0,"Among the 5 pillars of this religion is Salat,...",Islam,among the 5 pillars of this religion is salat ...,islam,rite on,among the 5 pillars of this religion is salat ...
9,3019,1997-10-16,Double Jeopardy!,STRAIT TALK,1000.0,"On July 3, 1988 an Iranian airliner enroute to...",the Strait of Hormuz,on july 3 1988 an iranian airliner enroute to ...,the strait of hormuz,strait talk,on july 3 1988 an iranian airliner enroute to ...


In [15]:
# questions = df['clean_question'].values
questions = df['everything'].values
# categories = df['clean_category'].values
# answers = df['clean_answer'].values

In [16]:
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in questions:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'questions', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [17]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [18]:
vocab_frame.head(20)

,words
the,the
number,numbers
on,on
a,a
man,mans
dress,dress
shirt,shirt
for,for
exampl,example
are,are


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib

# tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
#                                  min_df=0.2, stop_words='english',
#                                  use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))

# tfidf_vectorizer = TfidfVectorizer(stop_words='english',
#                                  tokenizer=tokenize_only, ngram_range=(1,3))

# %time tfidf_matrix = tfidf_vectorizer.fit_transform(questions)

tfidf_vectorizer.fit(questions)

# More memory efficient to dump the model and then load it
joblib.dump(tfidf_vectorizer, 'tfidf_test.pkl')

['tfidf_test.pkl', 'tfidf_test.pkl_01.npy', 'tfidf_test.pkl_02.npy']

In [20]:
tfidf_vectorizer = joblib.load('tfidf_test.pkl')
tfidf_matrix = tfidf_vectorizer.transform(questions)

In [21]:
terms = tfidf_vectorizer.get_feature_names()

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [23]:
from sklearn.cluster import KMeans

num_clusters = 50

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 4.47 s, sys: 64 ms, total: 4.53 s
Wall time: 2.7 s


In [27]:
df['Cluster'] = clusters

In [28]:
df['Cluster'].value_counts()

4     292
13     74
23     65
24     63
28     58
29     55
16     53
27     53
6      52
17     51
21     51
40     50
32     49
11     47
5      46
47     45
37     45
15     43
48     43
8      42
25     41
3      39
20     39
22     39
12     38
34     36
43     36
39     36
44     35
38     35
26     33
33     33
42     33
31     33
7      32
46     32
41     32
36     30
35     28
2      28
19     28
45     25
10     24
9      24
30     23
18     22
49     20
1      13
0      13
14     11
Name: Cluster, dtype: int64

In [39]:
from __future__ import print_function

cluster_words = []
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    cluster_words.append([])
    for ind in order_centroids[i, :5]: #replace 10 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
        cluster_words[i].append('%s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'))
    print() #add whitespace
    print() #add whitespace
    
#     print("Cluster %d categories:" % i, end='')
#     for cat in df.ix[i]['Category']#.values.tolist():
#         print(' %s,' % title, end='')
#     print() #add whitespace
#     print() #add whitespace
    
# print()
# print()

Top terms per cluster:

Cluster 0 words: pirate, gas, ship, verb, jason,

Cluster 1 words: sound, according, music, whistle, sound,

Cluster 2 words: century, 19th, 19th, women, married,

Cluster 3 words: historically, said, charles, hospital, people,

Cluster 4 words: ill, known, stars, middle, make,

Cluster 5 words: capitals, world, world, city, paris,

Cluster 6 words: city, races, newspapers, annual, college,

Cluster 7 words: years, species, animals, bird, kingdom,

Cluster 8 words: seen, radio, war, world, world,

Cluster 9 words: opened, town, house, massachusetts, miller,

Cluster 10 words: stone, elvis, female, mary, presleys,

Cluster 11 words: america, use, miss, word, precedes,

Cluster 12 words: saintly, born, history, peter, patron,

Cluster 13 words: film, titled, movie, asked, come,

Cluster 14 words: highest, income, mountain, highest, point,

Cluster 15 words: states, shared, oklahoma, official, mississippi,

Cluster 16 words: country, became, world, country, future,

In [40]:
len(cluster_words)

50

In [48]:
cluster_names = []

for item in cluster_words:
    string = ''
    for i in item:
        string += i
        string += ' '
    cluster_names.append(string)

In [49]:
cluster_names

['pirate gas ship verb jason ',
 'sound according music whistle sound ',
 'century 19th 19th women married ',
 'historically said charles hospital people ',
 'ill known stars middle make ',
 'capitals world world city paris ',
 'city races newspapers annual college ',
 'years species animals bird kingdom ',
 'seen radio war world world ',
 'opened town house massachusetts miller ',
 'stone elvis female mary presleys ',
 'america use miss word precedes ',
 'saintly born history peter patron ',
 'film titled movie asked come ',
 'highest income mountain highest point ',
 'states shared oklahoma official mississippi ',
 'country became world country future ',
 'means term phrase word origin ',
 'william sir astronomer discovering court ',
 'fun category prize mainly nobel ',
 'sea oil equals friends reserved ',
 'lines game ball services recognize ',
 'good home land ways away ',
 'song music stands chicago classic ',
 'word like early egg vaudeville ',
 'food treat drinking brains activi

### Visualization attempts

In [31]:
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from sklearn.manifold import MDS

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

In [56]:
cluster_dict = dict(zip(range(50), cluster_names))

In [57]:
cluster_dict

{0: 'pirate gas ship verb jason ',
 1: 'sound according music whistle sound ',
 2: 'century 19th 19th women married ',
 3: 'historically said charles hospital people ',
 4: 'ill known stars middle make ',
 5: 'capitals world world city paris ',
 6: 'city races newspapers annual college ',
 7: 'years species animals bird kingdom ',
 8: 'seen radio war world world ',
 9: 'opened town house massachusetts miller ',
 10: 'stone elvis female mary presleys ',
 11: 'america use miss word precedes ',
 12: 'saintly born history peter patron ',
 13: 'film titled movie asked come ',
 14: 'highest income mountain highest point ',
 15: 'states shared oklahoma official mississippi ',
 16: 'country became world country future ',
 17: 'means term phrase word origin ',
 18: 'william sir astronomer discovering court ',
 19: 'fun category prize mainly nobel ',
 20: 'sea oil equals friends reserved ',
 21: 'lines game ball services recognize ',
 22: 'good home land ways away ',
 23: 'song music stands chic

In [ ]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

In [60]:
#some ipython magic to show the matplotlib plots inline
%matplotlib inline 

#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys))#, label=clusters, title=titles)) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)

KeyError: 'label'

### LDA Attempts - work in progress

In [44]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

In [45]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

In [47]:
from gensim import corpora, models, similarities 

#remove proper names
%time preprocess = [strip_proppers(doc) for doc in questions]

#tokenize
%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

#remove stop words
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

CPU times: user 4.21 s, sys: 20 ms, total: 4.23 s
Wall time: 4.21 s
CPU times: user 9.58 s, sys: 76 ms, total: 9.65 s
Wall time: 9.57 s
CPU times: user 1.34 s, sys: 28 ms, total: 1.37 s
Wall time: 1.32 s


In [48]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [50]:
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

CPU times: user 28min 56s, sys: 1.22 s, total: 28min 57s
Wall time: 28min 57s


In [51]:
lda.show_topics()

[(0,
  u'0.011*citi + 0.011*state + 0.007*countri + 0.007*new + 0.007*name + 0.007*nation + 0.006*one + 0.006*us + 0.006*first + 0.005*world'),
 (1,
  u'0.009*film + 0.009*play + 0.006*titl + 0.005*one + 0.005*show + 0.004*star + 0.004*day + 0.004*song + 0.004*music + 0.004*man'),
 (2,
  u'0.009*one + 0.008*name + 0.008*war + 0.005*first + 0.005*presid + 0.004*henri + 0.003*king + 0.003*seen + 0.003*american + 0.003*year'),
 (3,
  u'0.014*name + 0.006*mean + 0.005*call + 0.005*island + 0.005*king + 0.004*like + 0.004*may + 0.004*one + 0.004*first + 0.003*use'),
 (4,
  u'0.007*type + 0.006*sea + 0.005*name + 0.005*use + 0.004*one + 0.004*clue + 0.004*first + 0.003*anim + 0.003*seen + 0.003*crew')]

In [67]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print [str(word) for word in i]
    print ''

ValueError: setting an array element with a sequence

In [79]:
for i in range(len(topics_matrix)):
    print i

0
1
2
3
4
